<a href="https://colab.research.google.com/github/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_NG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PBMC_NG dataset: Processes the BUG files into files prepared for use in R**

This notebook processes the output from the fastq file processing for this dataset. The data produced here is pre-generated and downloaded by the figure generation code. The purpose of this processing step is to prepare the data for figure generation, by filtering the data and producing downsampled datasets in addition to the original one. 

Steps:
1. Clone the code repo and download data to process
2. Prepare the R environment
3. Process the data
4. Generate statistics for the dataset

**1. Clone the code repo and download data to process**

In [1]:
![ -d "GRNP_2020" ] && rm -r GRNP_2020

!git clone https://github.com/pachterlab/GRNP_2020.git


Cloning into 'GRNP_2020'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 1099 (delta 145), reused 79 (delta 44), pack-reused 883
Receiving objects: 100% (1099/1099), 7.42 MiB | 18.57 MiB/s, done.
Resolving deltas: 100% (699/699), done.


In [2]:
#download BUG data from Zenodo
!mkdir data
!cd data && wget https://zenodo.org/record/3924675/files/PBMC_NG.zip?download=1 && unzip 'PBMC_NG.zip?download=1' && rm 'PBMC_NG.zip?download=1'

--2020-07-02 20:55:08--  https://zenodo.org/record/3924675/files/PBMC_NG.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375291643 (358M) [application/octet-stream]
Saving to: ‘PBMC_NG.zip?download=1’

PBMC_NG.zip?downloa 100%[===================>] 357.91M  12.8MB/s    in 25s     

2020-07-02 20:55:34 (14.5 MB/s) - ‘PBMC_NG.zip?download=1’ saved [375291643/375291643]

Archive:  PBMC_NG.zip?download=1
   creating: PBMC_NG/
   creating: PBMC_NG/bus_output/
  inflating: PBMC_NG/bus_output/bug.txt  
  inflating: PBMC_NG/bus_output/coll.genes.txt  
  inflating: PBMC_NG/bus_output/transcripts_to_genes.txt  


In [3]:
#Check that download worked
!cd data && ls -l && cd PBMC_NG/bus_output && ls -l

total 4
drwxr-xr-x 3 root root 4096 Jul  1 00:00 PBMC_NG
total 1765132
-rw-r--r-- 1 root root 1798406829 Jul  1 14:53 bug.txt
-rw-r--r-- 1 root root     738211 Jul  1 13:38 coll.genes.txt
-rw-r--r-- 1 root root    8335642 Jul  1 13:25 transcripts_to_genes.txt


**2. Prepare the R environment**

In [4]:
#switch to R mode
%reload_ext rpy2.ipython


/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [5]:
#install the R packages
%%R
install.packages("qdapTools")
install.packages("dplyr")
install.packages("stringdist")
install.packages("stringr")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘bitops’, ‘chron’, ‘data.table’, ‘RCurl’, ‘XML’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/bitops_1.0-6.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 8734 bytes

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

**3. Process the data**

Here we discard multimapped UMIs and all UMIs belonging to cells with fewer than 200 UMIs. We also precalculate gene expression, fraction of single-copy molecules etc. and save as stats (statistics). These can later be used when generating figures. We also generate down-sampled BUGs.

In [6]:
#create output directory
!mkdir figureData

In [7]:
#First set some path variables
%%R
source("GRNP_2020/RCode/pathsGoogleColab.R")


In [8]:
#Process and filter the BUG file
%%R
source(paste0(sourcePath, "BUGProcessingHelpers.R"))
createStandardBugsData(paste0(dataPath,"PBMC_NG/"), "PBMC_NG", c(0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1))



R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘qdapTools’


R[write to console]: The following object is masked from ‘package:dplyr’:

    id




[1] "Generating data for PBMC_NG"
[1] "Reading BUG from data/PBMC_NG/ ..."
[1] "Filtering multi-mapped reads..."
[1] "Fraction multi-mapped reads: 0.175510272359029"
[1] "Converting genes..."
[1] "Done"
[1] "Down-sampling in total 7 bugs:"
[1] "7: Down-sampling to 1"
[1] "saving BUG..."
[1] "creating stats..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "cpm normalizing..."
[1] "6: Down-sampling to 0.8"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



...

[1] "cpm normalizing..."
[1] "2: Down-sampling to 0.1"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "cpm normalizing..."
[1] "1: Down-sampling to 0.05"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "cpm normalizing..."
[1] "Done"
[1] "Saving stats..."


**4. Generate statistics for the dataset**

Here we create a file with various statistics for the dataset, which is used for generating table S2. It also contains some additional information about the dataset. Generation of this file may take several hours.

In [ ]:
%%R
source(paste0(sourcePath, "GenBugSummary.R"))
genBugSummary("PBMC_NG", "FGF23", "RPS10", 10)

[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 10

In [ ]:
!cd figureData/PBMC_NG && ls -l && cat ds_summary.txt

total 1209920
-rw-r--r-- 1 root root 310471361 Jul  1 23:01 Bug_100.RData
-rw-r--r-- 1 root root  63969118 Jul  1 22:48 Bug_10.RData
-rw-r--r-- 1 root root 114450505 Jul  1 22:49 Bug_20.RData
-rw-r--r-- 1 root root 189798037 Jul  1 22:52 Bug_40.RData
-rw-r--r-- 1 root root  34006703 Jul  1 22:47 Bug_5.RData
-rw-r--r-- 1 root root 241634831 Jul  1 22:55 Bug_60.RData
-rw-r--r-- 1 root root 282697430 Jul  1 22:58 Bug_80.RData
-rw-r--r-- 1 root root       989 Jul  2 01:36 ds_summary.txt
-rw-r--r-- 1 root root   1912011 Jul  1 23:01 Stats.RData
Dataset: PBMC_NG

totUMIs: 31378428
totCells: 5648
totCounts: 68838206
countsPerUMI: 2.19380671332547
UMIsPerCell: 5555.67067988669
countsPerCell: 12188.0676345609
totFracOnes: 0.43497937500247
FracMolWithUMIDistToNeighborH: 54, 607, 1183, 154, 2, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborL: 66, 567, 1178, 185, 4, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborHFrac: 0.027, 0.3035, 0.5915, 0.077, 0.001, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborLFrac: 0.0